# DSG: MultinomialNB

In [56]:
import numpy as np
import pandas as pd
import os

from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import cross_val_score, train_test_split,StratifiedKFold, StratifiedShuffleSplit
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from lib_DSG import ColumnSelector, DenseTransformer


folder = os.getcwd() ; print folder

/home/arda/Documents/youtube


# Importing data

In [122]:
pd_train = pd.read_csv('./data/train_sample_munged.csv', header=0, escapechar='\\', quotechar='"', low_memory=False)
pd_test = pd.read_csv('./data/test_sample_munged.csv', header=0, escapechar='\\', quotechar='"', low_memory=False)

In [121]:
pd_train['relevantTopicIds'] = pd_train['relevantTopicIds'].apply(lambda r: "".join(r) )
pd_train = pd_train['topicIds'].apply(lambda r: "".join(r))

In [125]:
pd_train

,video_category_id,title,description,published_at,viewCount,likeCount,dislikeCount,favoriteCount,commentCount,duration,dimension,definition,caption,licensedContent,topicIds,relevantTopicIds,dimension_2d,dimension_3d,definition_hd,definition_sd
0,19,Disneyland Railroad at night and first look at...,Disneyland Railroad at night and first look at...,2014-11-15 17:00:06,3826,154,0,0,26,482,2d,hd,0,1,"['/m/02fzs', '/m/0550hw', '/m/028k8w']","['/m/010jjr', '/m/01zx_2', '/m/09b3v', '/m/01j...",1,0,1,0
1,19,David Kills Goliath,NaN,2013-09-27 10:56:23,30961,69,3,0,7,131,2d,sd,0,0,"['/m/0cm45', '/m/02cvn']","['/m/015j7', '/m/06bvp']",1,0,0,1
2,19,Downtown Disney | Walt Disney World | Parques ...,Qué quieres hacer cuando no estás en uno de lo...,2014-05-27 21:57:18,64558,68,12,0,7,144,2d,hd,0,1,"['/m/081nh', '/m/09b1k']","['/m/09b3v', '/m/03tmlz', '/m/0166ls', '/m/04l...",1,0,1,0
3,19,The Renaissance Faire!,"This month, my mission for the Fiesta Movement...",2013-07-30 23:00:18,13776,662,6,0,193,273,2d,hd,0,1,['/m/04dlk_'],"['/m/04rlf', '/m/06cvx']",1,0,1,0
4,19,On board Quantum of the Seas - North Star | Ri...,Ich habe vom 31.10. - 02.11.2014 eine Kreuzfah...,2014-11-09 19:49:21,9233,57,0,0,11,306,2d,hd,0,1,"['/m/0gh8bm0', '/m/01tq2l', '/m/038qps', '/m/0...","['/m/06q74', '/m/07nrkh', '/m/04lm2k']",1,0,1,0
5,19,SHARMEEN PITHI DANCES 1,NaN,2014-03-20 08:02:45,209208,413,36,0,12,498,2d,hd,0,0,['/m/026bk'],"['/m/01chg', '/m/01gq53', '/m/081hv', '/m/02mt...",1,0,1,0
6,19,The CAR Carrier : Indian Railways Freight Wagon,Catch the specially compiled video of the 3 Ge...,2014-10-09 17:48:12,20586,109,4,0,42,300,2d,hd,0,1,"['/m/0303gp', '/m/06d_3', '/m/0k4j', '/m/07jdr']","['/m/07bsy', '/m/03x9c7', '/m/03rk0', '/m/07yv...",1,0,1,0
7,19,"Florence, Italy: Michelangelo's David",One of Europe's great thrills is seeing Michel...,2013-07-03 15:00:18,53171,180,4,0,22,156,2d,hd,1,1,"['/m/058w5', '/m/0953j', '/m/067s59']","['/m/031y2', '/m/03rjj', '/m/04lm2k']",1,0,1,0
8,19,Dubrovnik and Balkan Side-Trips,Nestled inside its mighty ramparts is Dubrovni...,2013-08-19 23:15:19,40431,146,5,0,11,1497,2d,hd,1,1,"['/m/067s59', '/m/0fbwq']","['/m/014dsx', '/m/04lm2k', '/m/07bxq', '/m/02p...",1,0,1,0
9,19,McShrimp and Egg Burger,McDonald's in other countries have the weirdes...,2014-06-24 18:10:03,485701,6808,357,0,715,199,2d,hd,0,0,"['/m/09syvmf', '/m/0cdn1']","['/m/07gyp7', '/m/033cnk', '/m/02wbm', '/m/02v...",1,0,1,0


In [85]:
pd_train = pd_train.fillna('')
pd_test = pd_test.fillna('')

In [86]:
print pd_train.columns

Index([u'video_category_id', u'title', u'description', u'published_at',
       u'viewCount', u'likeCount', u'dislikeCount', u'favoriteCount',
       u'commentCount', u'duration', u'dimension', u'definition', u'caption',
       u'licensedContent', u'topicIds', u'relevantTopicIds', u'dimension_2d',
       u'dimension_3d', u'definition_hd', u'definition_sd'],
      dtype='object')


In [97]:
a =pd_train['topicIds'].apply(lambda r: "".join(r))

# First model

In [105]:
dico_pattern={'match_lowercase_only':'\\b[a-z]+\\b',
              'match_word':'\\w{1,}',
              'match_word1': '(?u)\\b\\w+\\b',
              'match_word_punct': '\w+|[,.?!;]',
              'match_NNP': '\\b[A-Z][a-z]+\\b|\\b[A-Z]+\\b',
              'match_punct': "[,.?!;'-]"
             }
tfv_title = TfidfVectorizer(lowercase=True, stop_words='english', token_pattern=dico_pattern["match_word1"], 
                      ngram_range=(1, 2), max_df=1.0, min_df=2, max_features=None, 
                      vocabulary=None, binary=True, norm=u'l2', 
                      use_idf=True, smooth_idf=True, sublinear_tf=True)

tfv_desc = TfidfVectorizer(lowercase=True, stop_words='english', token_pattern=dico_pattern["match_word1"], 
                      ngram_range=(1, 2), max_df=1.0, min_df=2, max_features=None, 
                      vocabulary=None, binary=True, norm=u'l2', 
                      use_idf=True, smooth_idf=True, sublinear_tf=True)

clf = MultinomialNB(alpha=0.05, fit_prior=True, class_prior=None)

title_pipe = make_pipeline(ColumnSelector(key='title'), tfv_title)

desc_pipe = make_pipeline(ColumnSelector(key='description'), tfv_desc)

pipeline = make_union(title_pipe, desc_pipe)
pipeline.transformer_weights=[1.5,1]

In [106]:
Y = pd_train[u'video_category_id'].values
X = pipeline.fit_transform(pd_train) ; print X.shape
X_test = pipeline.transform(pd_test)

(239225, 594406)


In [123]:
alphas = np.arange(0.01, 0.2, 0.03)
#alphas = [0.06]
print alphas

[ 0.01  0.04  0.07  0.1   0.13  0.16  0.19]


In [124]:
results=[]
for i in alphas:
    clf.alpha = i

    skf = StratifiedShuffleSplit(Y, 10, test_size=0.2, random_state=0)
    scores_sss = cross_val_score(clf, X, Y,scoring='accuracy',cv=sss, n_jobs=-1)

    print ("SSS: acc: %0.4f, std: %0.4f, alpha: %s" %(scores_sss.mean(), scores_sss.std(), i))

TypeError: __init__() got an unexpected keyword argument 'test_size'

In [5]:
Y = pd_train['Author'].values
X = pipeline.fit_transform(pd_train)
X_test = pipeline.transform(pd_test)

print X.shape

 (28723, 77821)


# Final dump

# SUBMIT KAGGLE

In [103]:
clf.alpha = 0.01

In [45]:
y_pred = NB.predict(X_test)

In [46]:
submit = pd.DataFrame(index=None)
submit['Id']=pd_test['Id']
submit['Pred']=y_pred

In [47]:
submit.to_csv(folder+'/results/0.arda1.csv',sep=';',index=None)

# FINAL DUMP

In [101]:
clf.alpha = 0.01

In [102]:
final_pipe = make_pipeline(pipeline, clf)

In [ ]:
final_pipe.fit